# Linear Elasticity Problem with Mixed Element Method
This notebook treats the linear elasticity problem for materials with high values of the Lamé parameters ($\kappa \gg 1$).

## The PDE problem
The governing equations for small elastic deformations of a body $ \Omega $ can be expressed as:

$$
\begin{align}
    -\nabla \cdot \sigma(u) &= f & \text{in } \Omega \\
    \sigma(u) &= \kappa \text{tr}(\epsilon(u))I + 2 \mu \epsilon(u)
\end{align}
$$

where $ \sigma $ is the stress tensor, $ f $ represents the body force per unit volume, $ \kappa $ and $ \mu $ are Lamé's elasticity parameters for the material, $ I $ denotes the identity tensor, $ \epsilon $ is the symmetric strain tensor and the displacement vector field is denoted by $ u $. $ \epsilon := \frac{1}{2}(\nabla u + (\nabla u)^T) $



By substituting $ \epsilon(u) $ into $ \sigma $, we obtain:

$$
\sigma(u) = \kappa (\nabla \cdot u)I + \mu(\nabla u + (\nabla u)^T)
$$

Then, the strong formulation of linear elasticity is :
Find $ u \in V $ such that

$$
\begin{align}
    -\nabla \cdot \sigma (u) &= f & \text{in } & \Omega \\
    u &= 0 & \text{on } & \partial \Omega_D \\
    \sigma(u) \cdot n &= g_T & \text{on } & \partial \Omega_T
\end{align}
$$

where : 
- $ V = \{ v \in (H^1(\Omega))^3 : v = 0 \text{ on } \partial \Omega_D \} $,

- $ g_D $ is the Dirichlet boundary condition on the part $ \partial \Omega_D $ of the boundary,

- $ g_T $ is the traction vector on the part $ \partial \Omega_T $ of the boundary,

- $ n $ is the outward normal vector on the boundary $ \partial \Omega $,

- $ \partial \Omega_D \cap \partial \Omega_T = \emptyset $ and $ \partial \Omega = \partial \Omega_D \cup \partial \Omega_T $.

## The pressure field
In the case of high values of the Lamé parameter $ \kappa $, the pressure field $ p $ can be defined as:

$$
p := -\kappa \nabla \cdot u \hspace{1cm} \text{ in } \Omega
$$

This definition arises from the fact that, for isotropic materials, the stress tensor can be decomposed into hydrostatic and deviatoric components. The hydrostatic component is responsible for the pressure field, while the deviatoric component accounts for shear deformations.
As $u \in H^1(\Omega)$, the pressure field $p \in L^2(\Omega)$.

## The strong formulation of the problem
The strong formulation of the problem can be expressed as follows:
Find $ (u, p) \in V \times L^2(\Omega) $ such that
$$
\begin{align}
    -\nabla \cdot \sigma(u) &= f & \text{in } & \Omega \\
    u &= 0 & \text{on } & \partial \Omega_D \\
    \sigma(u) \cdot n &= g_T & \text{on } & \partial \Omega_T \\
    \nabla \cdot u + \frac{1}{\kappa} p &= 0 & \text{in } & \Omega
\end{align}
$$

## The weak formulation of the problem
To derive the weak formulation of the problem, we multiply the first equation by a test function $ v \in V $ and integrate by parts over the domain $ \Omega $ and we add the fourth equation multiplied by a test function $ q \in L^2(\Omega) $:

$$
\boxed{
\begin{aligned}
&\text{Find } (\vec u, p) \in V \times L^2(\Omega) \text{ such that:} \\
&\qquad \tilde{a}((\vec u, p), (\vec v, q)) = l(\vec v, q) \quad \forall (\vec v, q) \in V \times L^2(\Omega)
\end{aligned}
}
$$

with

$$
\begin{aligned}
&\tilde{a} : 
\begin{cases}
\left( V \times L^2(\Omega) \right)^2 \rightarrow \mathbb{R} \\
((\vec u, p), (\vec v, q)) \longmapsto \int_\Omega \left( 2\mu\, \boldsymbol{\varepsilon}(\vec u) : \boldsymbol{\varepsilon}(\vec v) - p\, \mathrm{div}\, \vec v + (\mathrm{div}\, \vec u)\, q + \frac{1}{\kappa} p q \right) \, dV
\end{cases} \\[0.3cm]
&l : 
\begin{cases}
V \rightarrow \mathbb{R} \\
\vec v \longmapsto \int_\Omega \vec f \cdot \vec v \, dV + \int_{\partial\Omega_N} \vec t_N \cdot \vec v \, dS
\end{cases}
\end{aligned}
$$

## Formal Model

In [ ]:
from sympde.expr import BilinearForm, LinearForm, integral
from sympde.expr     import find, EssentialBC, Norm, SemiNorm
from sympde.topology import VectorFunctionSpace, ScalarFunctionSpace, Cube, element_of
from sympde.calculus import grad, inner, div, Transpose
from sympde.core import Constant

from sympy import Tuple, sin, pi, cos
import numpy as np
import matplotlib.pyplot as plt

from psydac.api.discretization import discretize
from psydac.fem.basic          import FemField

domain = Cube()


kappa = Constant('kappa', real=True)
mu    = Constant('mu',    real=True)

V1 = VectorFunctionSpace('V1', domain,kind='H1')
V2 = ScalarFunctionSpace('V2', domain,kind='L2')
X = V1*V2

x,y,z = domain.coordinates

u,v = [element_of(V1, name=i) for i in ['u', 'v']]
p,q = [element_of(V2, name=i) for i in ['p', 'q']]

epsilon = lambda w: 0.5*(grad(w) + Transpose(grad(w)))

#bilinear form
a = BilinearForm(((u,p),(v,q)), integral(domain, inner(2*mu*epsilon(u),epsilon(v)) - p*div(v) + div(u)*q + (1/kappa)*p*q))

mu_val =1.
kappa_val =1.

# Exact solution
ue1 = 0
ue2 = 0
ue3 = sin(pi*x)*sin(pi*y)*sin(pi*z)
ue = Tuple(ue1, ue2, ue3)
pe = -kappa * pi * sin(pi*x) * sin(pi*y) * cos(pi*z)

#Second member
f1 = - pi*pi * (kappa + mu) *  cos(pi*x) * sin(pi*y) * cos(pi*z)
f2 = - pi*pi * (kappa + mu) *  sin(pi*x) * cos(pi*y) * cos(pi*z)
f3 =   pi*pi * (4*mu + kappa) * sin(pi*x) * sin(pi*y) * sin(pi*z)
f = Tuple(f1, f2, f3)

# linear form
l = LinearForm((v,q), integral(domain, inner(f, v)))

# Essential boundary conditions
bc = [EssentialBC(u, 0, domain.boundary)]

equation = find((u,p), forall=(v,q), lhs=a((u,p),(v,q)), rhs=l(v,q), bc=bc)

## Discretization

We shall need the **discretize** function from **PsyDAC**.

In [ ]:
d = 2 # discretization parameters
degree = [d,d,d]
h = 8
ncells = [h,h,h]

In [ ]:
# Create computational domain from topological domain
domain_h = discretize(domain, ncells=ncells, comm=None)

# Create discrete spline space
V1h = discretize(V1, domain_h, degree=degree)
V2h = discretize(V2, domain_h, degree=degree)
Xh = discretize(X,   domain_h, degree=degree)

# Discretize equation
equation_h = discretize(equation, domain_h, [Xh, Xh])

## Solving the PDE

In [ ]:
equation_h.set_solver('gmres', info=False, tol=1e-8)
phi_h = equation_h.solve(mu=mu_val, kappa=kappa_val)

## Computing the $L^2$ norm on $p$

In [ ]:
ph = FemField(V2h)
ph.coeffs[:] = phi_h.coeffs[3][:]

# create the formal Norm object
l2norm_p  = Norm(pe - p, domain, kind='l2')

# discretize the Norm object
l2norm_ph = discretize(l2norm_p, domain_h, V2h)

# assemble the norm
l2_error_p = l2norm_ph.assemble(p=ph,kappa=kappa_val, mu=mu_val)

print("L2 error in pressure:", l2_error_p)

## Computing the $L^2$ norm on $u$

In [ ]:
uh = FemField(V1h)
uh.coeffs[0][:] = phi_h.coeffs[0][:]
uh.coeffs[1][:] = phi_h.coeffs[1][:]
uh.coeffs[2][:] = phi_h.coeffs[2][:]

# create the formal Norm object
error_u   = [ue[0]-u[0], ue[1]-u[1], ue[2]-u[2]]
l2norm_u = Norm(error_u, domain, kind='l2')

# discretize the Norm object
l2norm_uh = discretize(l2norm_u, domain_h, V1h)

# assemble the Norm object
l2_error_u = l2norm_uh.assemble(u=uh, kappa=kappa_val, mu=mu_val)

print("L2 error in displacement:", l2_error_u)

## Computing the $H^1$ semi-norm on $u$

In [ ]:
# create the formal Norm object
h1_seminorm = SemiNorm(error_u, domain, kind='h1')

# discretize the norm
h1_seminorm_h = discretize(h1_seminorm, domain_h, V1h)

# assemble the norm
h1_semierror = h1_seminorm_h.assemble(u=uh)

print(h1_semierror)

## Computing the $H^1$ norm on $u$

In [ ]:
# create the formal Norm object
h1norm = Norm(error_u, domain, kind='h1')

# discretize the norm
h1norm_h = discretize(h1norm, domain_h, V1h)

# assemble the norm
h1_error = h1norm_h.assemble(u=uh)

print(h1_error)

## Plotting the simulation results, exact solution and error

In [ ]:
# %% plotting exact solution, simulation solution and error
import matplotlib.pyplot as plt
import numpy as np

h = 100
# Create a grid for plotting
x_vals = np.linspace(0, 1, h)
y_vals = np.linspace(0, 1, h)
z_vals = np.linspace(0, 1, h)
X, Y, Z = np.meshgrid(x_vals, y_vals, z_vals)
x_lim = 1
y_lim = 1
z_lim = 1

# Calculate the exact solution on the grid
exact_u1 = np.zeros_like(X)
exact_u2 = np.zeros_like(Y)
exact_u3 = np.sin(np.pi * X) * np.sin(np.pi * Y) * np.sin(np.pi * Z)
exact_p = -kappa_val * np.pi * np.sin(np.pi * X) * np.sin(np.pi * Y) * np.cos(np.pi * Z)

uh_sim = Xh.eval_field(phi_h, x_vals, y_vals, z_vals)
u1 = uh_sim[0]
u2 = uh_sim[1]
u3 = uh_sim[2]

# Calculate the error
error_u1 = u1 - exact_u1
error_u2 = u2 - exact_u2
error_u3 = u3 - exact_u3

# Displacement plots
z_plane_list = np.linspace(0, 1, 5)  #List of z-planes to plot
fig = plt.figure(figsize=(24, 6 * len(z_plane_list)))

for i, z_plane in enumerate(z_plane_list):
    list_index = int(z_plane * (h - 1))  # Index for the z-plane

    # Simulated u3
    ax1 = fig.add_subplot(len(z_plane_list), 3, 3 * i + 1, projection='3d')
    ax1.plot_surface(X[:, :, list_index], Y[:, :, list_index], u3[:, :, list_index], cmap='viridis')
    ax1.set_title('Simulated u3 on z={:.2f} plane'.format(z_plane))
    ax1.set_xlabel('X-axis')
    ax1.set_ylabel('Y-axis')
    ax1.set_zlabel('u3_simulated')
    ax1.set_xlim(0, x_lim)
    ax1.set_ylim(0, y_lim)
    ax1.set_zlim(-z_lim, z_lim)

    # Exact u3
    ax2 = fig.add_subplot(len(z_plane_list), 3, 3 * i + 2, projection='3d')
    ax2.plot_surface(X[:, :, list_index], Y[:, :, list_index], exact_u3[:, :, list_index], cmap='viridis')
    ax2.set_title('Exact u3 on z={:.2f} plane'.format(z_plane))
    ax2.set_xlabel('X-axis')
    ax2.set_ylabel('Y-axis')
    ax2.set_zlabel('u3_exact')
    ax2.set_xlim(0, x_lim)
    ax2.set_ylim(0, y_lim)
    ax2.set_zlim(-z_lim, z_lim)

    # Error in u3
    ax3 = fig.add_subplot(len(z_plane_list), 3, 3 * i + 3, projection='3d')
    ax3.plot_surface(X[:, :, list_index], Y[:, :, list_index], error_u3[:, :, list_index], cmap='viridis')
    ax3.set_title('Error in u3 on z={:.2f} plane'.format(z_plane))
    ax3.set_xlabel('X-axis')
    ax3.set_ylabel('Y-axis')
    ax3.set_zlabel('Error in u3')
    ax3.set_xlim(0, x_lim)
    ax3.set_ylim(0, y_lim)
    ax3.set_zlim(error_u3[:, :, list_index].min(), error_u3[:, :, list_index].max())
    mappable = ax3.plot_surface(X[:, :, list_index], Y[:, :, list_index], error_u3[:, :, list_index], cmap='viridis')
    fig.colorbar(mappable, ax=ax3, shrink=0.5, aspect=5)

plt.tight_layout()
plt.show()

In [ ]:
def compute_divergence(u1, u2, u3, x, y, z):
	du1_dx = np.gradient(u1, x, axis=0)
	du2_dy = np.gradient(u2, y, axis=1)
	du3_dz = np.gradient(u3, z, axis=2)
	return du1_dx + du2_dy + du3_dz

p_sim = - kappa_val * compute_divergence(u1, u2, u3, x_vals, y_vals, z_vals)

# Pressure plots
fig, ax = plt.subplots(1, 3, figsize=(18, 6))
# Simulated pressure
c1 = ax[0].contourf(X[:, :, list_index], Y[:, :, list_index], p_sim[:, :, list_index], cmap='viridis')
ax[0].set_title('Simulated Pressure on z={:.2f} plane'.format(z_plane))
ax[0].set_xlabel('X-axis')
ax[0].set_ylabel('Y-axis')
ax[0].set_xlim(0, x_lim)
ax[0].set_ylim(0, y_lim)
fig.colorbar(c1, ax=ax[0], shrink=0.5, aspect=5)

# Exact pressure
c2 = ax[1].contourf(X[:, :, list_index], Y[:, :, list_index], exact_p[:, :, list_index], cmap='viridis')
ax[1].set_title('Exact Pressure on z={:.2f} plane'.format(z_plane))
ax[1].set_xlabel('X-axis')
ax[1].set_ylabel('Y-axis')
ax[1].set_xlim(0, x_lim)
ax[1].set_ylim(0, y_lim)
fig.colorbar(c2, ax=ax[1], shrink=0.5, aspect=5)

# Error in pressure
c3 = ax[2].contourf(X[:, :, list_index], Y[:, :, list_index], p_sim[:, :, list_index] - exact_p[:, :, list_index], cmap='viridis')
ax[2].set_title('Error in Pressure on z={:.2f} plane'.format(z_plane))
ax[2].set_xlabel('X-axis')
ax[2].set_ylabel('Y-axis')
ax[2].set_xlim(0, x_lim)
ax[2].set_ylim(0, y_lim)
fig.colorbar(c3, ax=ax[2], shrink=0.5, aspect=5)


In [ ]:
p_simulated = -kappa_val * compute_divergence(u1, u2, u3, x_vals, y_vals, z_vals)
exact_p = -kappa_val * np.pi * np.sin(np.pi * X) * np.sin(np.pi * Y) * np.cos(np.pi * Z)
error_p = p_simulated - exact_p

fig = plt.figure(figsize=(24, 6 * len(z_plane_list)))
for i, z_plane in enumerate(z_plane_list):
    list_index = int(z_plane * (h - 1))

    # Simulated pressure
    ax1 = fig.add_subplot(len(z_plane_list), 3, 3 * i + 1, projection='3d')
    ax1.plot_surface(X[:, :, list_index], Y[:, :, list_index], p_simulated[:, :, list_index], cmap='viridis')
    ax1.set_title('Simulated Pressure on z={:.2f} plane'.format(z_plane))
    ax1.set_xlabel('X-axis')
    ax1.set_ylabel('Y-axis')
    ax1.set_zlabel('p_simulated')
    ax1.set_xlim(0, x_lim)
    ax1.set_ylim(0, y_lim)
    ax1.set_zlim(p_simulated[:, :, list_index].min(), p_simulated[:, :, list_index].max())

    # Exact pressure
    ax2 = fig.add_subplot(len(z_plane_list), 3, 3 * i + 2, projection='3d')
    ax2.plot_surface(X[:, :, list_index], Y[:, :, list_index], exact_p[:, :, list_index], cmap='viridis')
    ax2.set_title('Exact Pressure on z={:.2f} plane'.format(z_plane))
    ax2.set_xlabel('X-axis')
    ax2.set_ylabel('Y-axis')
    ax2.set_zlabel('p_exact')
    ax2.set_xlim(0, x_lim)
    ax2.set_ylim(0, y_lim)
    ax2.set_zlim(exact_p[:, :, list_index].min(), exact_p[:, :, list_index].max())

    # Error in pressure
    ax3 = fig.add_subplot(len(z_plane_list), 3, 3 * i + 3, projection='3d')
    mappable = ax3.plot_surface(X[:, :, list_index], Y[:, :, list_index], error_p[:, :, list_index], cmap='viridis')
    ax3.set_title('Error in Pressure on z={:.2f} plane'.format(z_plane))
    ax3.set_xlabel('X-axis')
    ax3.set_ylabel('Y-axis')
    ax3.set_zlabel('Error in pressure')
    ax3.set_xlim(0, x_lim)
    ax3.set_ylim(0, y_lim)
    ax3.set_zlim(error_p[:, :, list_index].min(), error_p[:, :, list_index].max())
    fig.colorbar(mappable, ax=ax3, shrink=0.5, aspect=5)

plt.tight_layout()
plt.show()
